In [1]:
import tensorflow as tf
import numpy as np
import os,glob,cv2
import sys,argparse

def getClassValue(result):
    if result >= 0 and result <= 9:
        return str(result)
    elif result >= 10 and result <= 35:
        return chr(65+result-10)
    else :
        return chr(97+result-36)

# First, pass the path of the image 
filename = 'C:/Users/shivangi pandey/Desktop/MinorProject/resized_dataset/Sample002/img002-00041 (2).png'
files = os.listdir('C:/Users/shivangi pandey/Desktop/MinorProject/testImg')
image_size=32
num_channels=1
images = []
# Reading the image using OpenCV

for file in files:
    image = cv2.imread('C:/Users/shivangi pandey/Desktop/MinorProject/testImg/'+file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#print(image.shape)
# Resizing the image to our desired size and preprocessing will be done exactly as done during training
#image = cv2.resize(image, (image_size, image_size),0,0, cv2.INTER_LINEAR)
    images.append(image)
    
images = np.array(images, dtype=np.uint8)
images = images.astype('float32')
images = np.multiply(images, 1.0/255.0) 
#The input to the network is of shape [None image_size image_size num_channels]. Hence we reshape.
x_batch = images.reshape(images.shape[0], image_size,image_size,num_channels)

## Let us restore the saved model 
sess = tf.Session()
# Step-1: Recreate the network graph. At this step only graph is created.
saver = tf.train.import_meta_graph('my_text_recognizer_model.meta')
# Step-2: Now let's load the weights saved using the restore method.
saver.restore(sess, tf.train.latest_checkpoint('./'))

# Accessing the default graph which we have restored
graph = tf.get_default_graph()

# Now, let's get hold of the op that we can be processed to get the output.
# In the original network y_pred is the tensor that is the prediction of the network
y_pred = graph.get_tensor_by_name("train_prediction:0")

## Let's feed the images to the input placeholders
x= graph.get_tensor_by_name("tf_train_dataset:0") 
y_true = graph.get_tensor_by_name("tf_train_labels:0") 
y_test_images = np.zeros((1, 62)) 


### Creating the feed_dict that is required to be fed to calculate y_pred 
feed_dict_testing = {x: x_batch, y_true: y_test_images}
result=sess.run(y_pred, feed_dict=feed_dict_testing)
# result is of this format [probabiliy_of_rose probability_of_sunflower]
#print(result)
#print(result.max())
#print(result.argmax())
#for r in result:
#   print(getClassValue(result.argmax()))


c:\users\shivangi pandey\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


INFO:tensorflow:Restoring parameters from ./my_text_recognizer_model


In [ ]:


i = 0
for r in result:
    char = getClassValue(r.argmax())
    cv2.namedWindow(char,cv2.WINDOW_NORMAL)
    cv2.resizeWindow(char, 200,150)
    cv2.imshow(char,images[i])
    cv2.waitKey(0)
    i += 1
    